In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("download.default_directory=" + os.getcwd() + "/downloads")

In [3]:
#Use pandas create dataframe (df) object that will store the data
columns = ['name','paper_type','citation','url','abstract','title','collaborators']
citations = pd.DataFrame()
#Place the list of names generated previously in the project into the df object
citations

""



Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [4]:
def google_scholar_scraper(record):
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(10) # seconds loading implicitly
    google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
    #Set driver to go to start_page and search a given name with valid scholar profile
    driver.get(google_scholar)
    #Instatiate the webdriver and set to implicitly wait for page objects to load

    profile_found = False
    
    search_bar = driver.find_element_by_class_name('gs_in_txt')    
    search_bar.send_keys("\"" + record.faculty + "\"" + " CSUN")
    search_bar.send_keys(Keys.ENTER)
    
    if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
        profile_found = True
    else:
        search_bar = driver.find_element_by_class_name('gs_in_txt')
        search_bar.clear()
        time.sleep(1)
        search_bar.send_keys("\"" + record.faculty + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
            profile_found = True
        else:
            driver.close()
        
    if profile_found:
        #Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
        view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a')
        view_profile_link.click() 

        # To get all the links we need to click the Show More multiple times to show the whole links 

        gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

        while not gs_bpf_wrapper.get_attribute("disabled"):
            gs_bpf_wrapper.click()
            time.sleep(3)

        #pull the first citation for the page
        view_citation_links = driver.find_elements_by_class_name('gsc_a_at')
        #print(view_citation_links)

        citations = []
        patents = []
        other = []

        citation_file = open('google_scholar_citations.txt', "a")
        patent_file = open('google_scholar_patents.txt', "a")

        for link in view_citation_links:
            
            not_his_paper = False

            title = link.text

            citation = {
            "Authors" : "",
            "Title" : "",
            "Date" : "",
            "Publisher": "",
            "Description": "",
            "Journal": "",
            "Issue": "",
            "Volume":"",
            "Pages": ""
            }

            patent = {
            "Inventors" : "",
            "Title" : "",
            "Date" : "",
            "Patent Office": "",
            "Description": "",
            "Application Number": "",
            "Patent Number": ""
            }
            inventors = ""

            link.click()

            citation_table = driver.find_element_by_id("gsc_vcd_table")
            citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")

            data_category = citation_table_data_objects[0].find_element_by_tag_name("div")

            if data_category.text == "Authors":

                i = 0
                for data in citation_table_data_objects:

                    data_category = citation_table_data_objects[i].find_element_by_tag_name("div")

                    citation_title = driver.find_element_by_id('gsc_vcd_title')
                    citation["Title"] = citation_title.text

                    if data_category.text == "Authors":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Authors"] = data_field.text
                        if (not record.faculty in data_field.text) or ():
                            not_his_paper = True
                    elif data_category.text == "Publication date":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Date"] = data_field.text
                    elif data_category.text == "Publisher":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Publisher"] = data_field.text
                    elif data_category.text == "Description":
                        # this line of code grabs the description
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        
                        # here is where we want to go through the description and
                        # replace every instance of "\n" with " "
                        
                        # saves the description into the dictionary
                        citation["Description"] = data_field.text
                    elif data_category.text == "Journal":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Journal"] = data_field.text
                    elif data_category.text == "Issue":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Issue"] = data_field.text
                    elif data_category.text == "Volume":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Volume"] = data_field.text
                    elif data_category.text == "Pages":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        citation["Pages"] = data_field.text
                    i = i + 1
                    
                #if not not_his_paper:
                
                # remember to re-indent into the if statement
                citation_record = record.faculty + "|" + citation['Title'] + "|" + citation['Authors'] + "|" + citation['Date'] + "|" + citation['Journal'] + "|" + citation['Volume'] + "|" + citation['Issue'] + "|" + citation['Pages'] + "|" + citation['Publisher'] + "|" + citation['Description'] + "\n"
                citation_file.write(citation_record)

            elif data_category.text == "Inventors":

                for data in citation_table_data_objects:

                    i = 0

                    data_category = citation_table_data_objects[i].find_element_by_tag_name("div")

                    citation_title = driver.find_element_by_id('gsc_vcd_title')
                    citation["Title"] = citation_title.text

                    if data_category.text == "Inventors":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        inventors = data_field.text
                    elif data_category.text == "Publication date":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        patent["Date"] = data_field.text
                    elif data_category.text == "Patent office":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        patent["Patent Office"] = data_field.text
                    elif data_category.text == "Description":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        patent["Description"] = data_field.text
                    elif data_category.text == "Application number":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        patent["Application Number"] = data_field.text
                    elif data_category.text == "Patent number":
                        data_field = data.find_element_by_class_name("gsc_vcd_value")
                        patent["Patent Number"] = data_field.text

                    # this is where we would store this record in a pandas dataframe
                    i += 1
                    
                #if not not_his_paper:
                
                # remember to re-indent into the if statement
                patent_record = record.faculty + "|" + patent['Title'] + "|" + patent['Inventors'] + "|" + patent['Date'] + "|" + patent['Patent Office'] + "|" + patent['Description'] + "|" + patent['Application Number'] + "|" + patent['Patent Number'] + "\n"
                patent_file.write(patent_record)


            elif not_his_paper:
                print("Error: Publication ", i , " did not get captured by program; new decision tree required.")

            close_table = driver.find_element_by_id("gs_md_cita-d-x")     
            close_table.click()
            time.sleep(3)

        citation_file.close()
        patent_file.close()    
        driver.close()
    else:
        f = open("google_scholar_no_results.csv", "a")
        f.write(record.faculty + ",")
        f.write(record.college + ",")
        f.write(record.department + ",")
        f.write(record.url + "\n")
        f.close()

In [5]:
view_content = pd.read_csv("view_content.csv")
view_content.head()

,faculty,college,department,url
0,Sandor Samuels,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
1,Kurt Saunders,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
2,Hilary Silvia-Goldberg,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
3,Melanie Williams,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
4,Gregory Young,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...


In [10]:
view_content.iloc[95]

faculty                                        Nanci Carr 
college                  College of Business and Economics
department                      Department of Business Law
url            https://www.csun.edu/blaw/faculty-directory
Name: 95, dtype: object

In [11]:
google_scholar_scraper(view_content.iloc[95])

In [7]:
for i in range(96,view_content.shape[0]):
    google_scholar_scraper(view_content.iloc[i])

In [29]:
data = "My name is Jorge Martinez"

In [30]:
not_his_paper = True
if "Jorge" in data:
    not_his_paper = False
print(not_his_paper)

False
